In [6]:
import pycountry
import pandas as pd
import numpy as np 

df = pd.read_csv("../data/covid_confirmed_cases.csv").melt(id_vars=['Province/State','Country/Region','Lat','Long'], var_name='date', value_name='cases').drop('Province/State', axis=1)
df['date'].apply(pd.to_datetime)
df=df.sort_values(by='date')
df.head()

keys = [list(pycountry.countries)[i].alpha_3 for i in range(len(pycountry.countries))]
names = [list(pycountry.countries)[i].name for i in range(len(pycountry.countries))]
conv = { v:k for k,v in zip(keys,names) }
def match_name(x):
    try:
        return conv[x]
    except KeyError as e:
        for n in names:
            if x in n:
                return conv[n]
        if 'US' in x:
            return conv['United States']
        return 'N/A'
df['codes'] = df.apply(lambda x: match_name(x['Country/Region']), axis=1)

df.head()

,Country/Region,Lat,Long,date,cases,codes
0,Thailand,15.0000,101.0000,1/22/20,2,THA
276,US,35.1269,-89.9253,1/22/20,0,USA
275,US,33.9533,-117.3961,1/22/20,0,USA
274,US,34.0132,-85.1479,1/22/20,0,USA
273,US,30.5773,-86.6611,1/22/20,0,USA


In [12]:
import plotly.graph_objects as go
dates = df['date'].unique()
dates_sliders = []

for date in dates:
    tdf = df[df['date'] == date]

    dates_sliders.append(go.Choropleth(
        visible=False,
        locations = tdf['codes'],
        z = tdf['cases'],
        text = tdf['Country/Region'],
        colorscale = 'RdBu',
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = 'Cases'
    ))

fig = go.Figure(data=dates_sliders)

steps = []
for i in range(len(dates)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Date: " + str(dates[i])}],
              label=str(dates[i])  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=len(dates),
    currentvalue={"prefix": "Steps: "},
    pad={"t": 50},
    steps=steps,
)]

geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='orthographic'
    )
fig.update_layout(
    sliders=sliders,
    geo=geo
)
fig.data[0].visible = True
fig.write_html('../plots/simple_globe_covid.html')

fig.show()

In [28]:
import plotly.express as px
tdf = df.copy(deep=True)
tdf = tdf[tdf['date'] > '3/2/2020']
tdf['max_cases'] = [(v if v<1000 else 1000) for v in tdf['cases']]
fig = px.scatter_geo(tdf, locations='codes',
                     hover_name="Country/Region", size="max_cases",
                     projection="natural earth", animation_frame="date",)
fig.show()
fig.write_html("../plots/testing_covid_cases_flat.html")

In [27]:
import plotly.express as px
tdf = df.copy(deep=True)
tdf = tdf[tdf['date'] > '3/2/2020']
tdf['max_cases'] = [(v if v<1000 else 1000) for v in tdf['cases']]
fig = px.scatter_geo(tdf, locations='codes',
                     hover_name="Country/Region", size="max_cases",
                     projection="orthographic", animation_frame="date",)
fig.show()
fig.write_html("../plots/round_covid_cases.html")